In [1]:
import keras_nlp

# Load the model architecture
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [2]:
# Path to your weights file
weights_path = "/kaggle/input/gemma-quizard-weights/keras/default/1/QA-Gemma-Quizard.weights.h5"

# Load the weights
gemma_lm.load_weights(weights_path)

/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 210 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

In [4]:
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)

In [5]:
template = """Instruction: Generate an answer to the question using the provided context.  Provide the answer only without including any additional content."

Context:
{Context}

Question:
{Question}

Response:
{Answer}
"""

In [6]:
prompt = template.format(
    Context="The Great Barrier Reef is the world's largest coral reef system, located in the Coral Sea, off the coast of Queensland, Australia. It is composed of over 2,900 individual reefs and 900 islands stretching over 2,300 kilometers. The reef is known for its biodiversity, hosting countless marine species, and is a popular destination for snorkeling and diving enthusiasts. However, it faces threats from climate change, overfishing, and pollution.",
    Question="What are some of the major threats faced by the Great Barrier Reef?",
    Answer=""
)

sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

I0000 00:00:1735126711.185662      79 service.cc:145] XLA service 0x7e2c1ee06950 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1735126711.187009      79 service.cc:153]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1735126711.603908      79 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Instruction: Generate an answer to the question using the provided context.  Provide the answer only without including any additional content."

Context:
The Great Barrier Reef is the world's largest coral reef system, located in the Coral Sea, off the coast of Queensland, Australia. It is composed of over 2,900 individual reefs and 900 islands stretching over 2,300 kilometers. The reef is known for its biodiversity, hosting countless marine species, and is a popular destination for snorkeling and diving enthusiasts. However, it faces threats from climate change, overfishing, and pollution.

Question:
What are some of the major threats faced by the Great Barrier Reef?

Response:

The major threats to the Great Barrier Reef include climate change, overfishing, and pollution, which have led to coral bleaching and habitat degradation, impacting its biodiversity and ecological balance.

"


In [7]:
prompt = template.format(
    Context="After training a machine translation model on a large dataset, the BLEU score was calculated to evaluate the quality of the model's translations compared to a set of human-generated reference translations. The model's output was compared to the reference texts using n-gram overlap, and a brevity penalty was applied to avoid overly short translations. The BLEU score was found to be 0.75, which indicates that the model's translations closely align with the reference texts.",
    Question="What is the BLEU score of the model's output as calculated during evaluation?",
    Answer=""
)

sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction: Generate an answer to the question using the provided context.  Provide the answer only without including any additional content."

Context:
After training a machine translation model on a large dataset, the BLEU score was calculated to evaluate the quality of the model's translations compared to a set of human-generated reference translations. The model's output was compared to the reference texts using n-gram overlap, and a brevity penalty was applied to avoid overly short translations. The BLEU score was found to be 0.75, which indicates that the model's translations closely align with the reference texts.

Question:
What is the BLEU score of the model's output as calculated during evaluation?

Response:

The BLEU score was 0.75, indicating that the model's translations were reasonably accurate and aligned with the reference texts.\


In [8]:
from kaggle_secrets import UserSecretsClient

# Retrieve the Hugging Face API token from Kaggle secrets
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")


In [9]:
!pip install PyPDF2 sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.8 MB/s eta 0:00:00


In [11]:
import PyPDF2
import re
import nltk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np
import keras_nlp

# Download the Punkt tokenizer for sentence splitting (only need to run once)
nltk.download('punkt')

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                text += page.extract_text() or ""
        return text
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        return ""

# Function to exclude references section
def exclude_references(text):
    match = re.search(r'(References|Bibliography|Works Cited)', text, re.IGNORECASE)
    if match:
        text = text[:match.start()]
    return text

# Function to split text into sentences
def split_text_into_sentences(text):
    return nltk.tokenize.sent_tokenize(text)

# Load the pre-trained model and tokenizer for question generation
question_model_name = "aayeshanakarmi/T5-QG-finetuned-squad"
question_tokenizer = AutoTokenizer.from_pretrained(question_model_name, use_auth_token=hf_token)
question_model = AutoModelForSeq2SeqLM.from_pretrained(question_model_name, use_auth_token=hf_token)

# Load SentenceTransformer model for semantic similarity
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

# Load the Gemma model for answer generation
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")
weights_path = "/kaggle/input/gemma-quizard-weights/keras/default/1/QA-Gemma-Quizard.weights.h5"  # Update to actual path
gemma_lm.load_weights(weights_path)

# Configure the sampler for the Gemma model
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)

# Template for answer generation
answer_template = """Instruction: Generate an answer to the question using the provided context. Provide the answer only without including any additional content.

Context:
{Context}

Question:
{Question}

Response:
"""

# Function to generate questions from a group of sentences
def generate_questions_from_group(sentence_group):
    paragraph = " ".join(sentence_group)
    inputs = question_tokenizer(paragraph, return_tensors="pt", truncation=True, max_length=512)
    outputs = question_model.generate(inputs['input_ids'], max_length=150, num_beams=4, early_stopping=True)
    question = question_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    if not question.strip():
        question = "No valid question generated."
    
    return question

# Function to generate answers using the Gemma model
def generate_answer_with_gemma(context, question):
    prompt = answer_template.format(Context=context, Question=question)
    response = gemma_lm.generate(prompt, max_length=256)
    return response  # Return the generated answer

# Function to generate unique questions and answers
def generate_unique_questions_and_answers(sentences):
    # Create embeddings for all sentences once
    embeddings = sentence_model.encode(sentences)
    qa_pairs = []

    # Group sentences based on similarity and generate questions and answers
    for i in range(len(embeddings)):
        current_group = [sentences[i]]
        for j in range(i + 1, len(embeddings)):
            similarity = cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]
            if similarity > 0.7:
                current_group.append(sentences[j])
        
        # Generate question and answer if the group is large enough
        if len(current_group) > 1:
            question = generate_questions_from_group(current_group)
            context = " ".join(current_group)
            answer = generate_answer_with_gemma(context, question)
            qa_pairs.append((question, answer, context))
    
    return qa_pairs

# Example usage
pdf_path = "/kaggle/input/attention-is-all-you-need/7181-attention-is-all-you-need.pdf"

# Step 1: Extract text from PDF
pdf_text = extract_text_from_pdf(pdf_path)

# Step 2: Exclude the references section
pdf_text = exclude_references(pdf_text)

# Step 3: Split the text into sentences
sentences = split_text_into_sentences(pdf_text)

# Step 4: Generate unique questions and answers based on context relevance
qa_pairs = generate_unique_questions_and_answers(sentences)

# Step 5: Save generated questions and answers to a file or print them
output_file = "/kaggle/working/generated_questions_and_answers_with_gemma.txt"
with open(output_file, "w") as f:
    for i, (question, answer, context) in enumerate(qa_pairs):
        f.write(f"Question {i+1}: {question}\n")
        f.write(f"Answer {i+1}: {answer}\n")
        f.write(f"Context {i+1}: {context}\n\n")

print(f"{len(qa_pairs)} question-answer pairs generated and saved to '{output_file}'.")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:796: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 210 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [13]:
!pip install torch transformers optimum

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from optimum.quantization import QuantizationConfig, quantize

# # Option 1: 4-bit Quantization
# def load_quantized_gemma(model_name="google/gemma-2b-it"):
#     # Load tokenizer
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
    
#     # Quantization configuration
#     quantization_config = QuantizationConfig(
#         load_in_4bit=True,
#         bnb_4bit_compute_dtype=torch.float16,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_use_double_quant=True
#     )
    
#     # Load model with quantization
#     model = AutoModelForCausalLM.from_pretrained(
#         model_name,
#         quantization_config=quantization_config,
#         device_map="auto"  # Automatic device placement
#     )
    
#     return model, tokenizer

# # Option 2: 8-bit Quantization
# def load_8bit_quantized_gemma(model_name="google/gemma-2b-it"):
#     # Load tokenizer
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
    
#     # Load model in 8-bit
#     model = AutoModelForCausalLM.from_pretrained(
#         model_name,
#         load_in_8bit=True,
#         device_map="auto"
#     )
    
#     return model, tokenizer

# # Option 3: Dynamic Quantization (works on CPU)
# def load_dynamically_quantized_gemma(model_name="google/gemma-2b-it"):
#     # Load tokenizer
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
    
#     # Load full precision model
#     model = AutoModelForCausalLM.from_pretrained(model_name)
    
#     # Apply dynamic quantization
#     model = torch.quantization.quantize_dynamic(
#         model, 
#         {torch.nn.Linear},  # Quantize linear layers
#         dtype=torch.qint8
#     )
    
#     return model, tokenizer

In [ ]:
import os
import gc
import csv
import re
import nltk
import torch
import numpy as np
import pandas as pd
import PyPDF2
from tqdm import tqdm

# Memory-efficient imports
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Download NLTK resources
nltk.download('punkt', quiet=True)

class MemoryEfficientQAGenerator:
    def __init__(self, 
                 pdf_path, 
                 question_model_name="aayeshanakarmi/T5-QG-finetuned-squad", 
                 sentence_model_name='all-MiniLM-L6-v2'):
        # Initialize with minimal memory usage
        self.pdf_path = pdf_path
        
        # Use CPU to reduce memory overhead
        self.device = torch.device('cpu')
        
        # Load models with minimal memory
        self.question_tokenizer = AutoTokenizer.from_pretrained(
            question_model_name, 
            low_cpu_mem_usage=True
        )
        self.question_model = AutoModelForSeq2SeqLM.from_pretrained(
            question_model_name, 
            low_cpu_mem_usage=True,
            device_map='cpu'
        )
        
        # Lightweight sentence transformer
        self.sentence_model = SentenceTransformer(
            sentence_model_name, 
            device=str(self.device)
        )
        
        # Memory-efficient configurations
        self.max_sentences = 500  # Limit sentences to process
        self.similarity_threshold = 0.6  # Adjustable similarity threshold
    
    def extract_text_from_pdf(self):
        """Memory-efficient PDF text extraction"""
        try:
            with open(self.pdf_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                text = ""
                for page in reader.pages:
                    text += page.extract_text() or ""
                return self._preprocess_text(text)
        except Exception as e:
            print(f"PDF extraction error: {e}")
            return ""
    
    def _preprocess_text(self, text):
        """Advanced text preprocessing"""
        # Remove references section
        ref_pattern = r'(References|Bibliography|Works Cited).*'
        text = re.sub(ref_pattern, '', text, flags=re.IGNORECASE | re.DOTALL)
        
        # Additional cleaning
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def split_sentences(self, text):
        """Memory-efficient sentence splitting"""
        sentences = nltk.sent_tokenize(text)
        return sentences[:self.max_sentences]  # Limit sentences
    
    def generate_questions(self, sentence_group):
        """Generate questions with memory constraints"""
        try:
            paragraph = " ".join(sentence_group)
            inputs = self.question_tokenizer(
                paragraph, 
                return_tensors="pt", 
                truncation=True, 
                max_length=512
            )
            
            outputs = self.question_model.generate(
                inputs['input_ids'], 
                max_length=150, 
                num_beams=4, 
                early_stopping=True
            )
            
            question = self.question_tokenizer.decode(
                outputs[0], 
                skip_special_tokens=True
            )
            
            return question.strip() or "No valid question generated."
        except Exception as e:
            print(f"Question generation error: {e}")
            return "Question generation failed."
    
    def generate_context_qa_pairs(self, sentences):
        """Generate context-based Q&A pairs with memory efficiency"""
        # Create embeddings with controlled memory
        embeddings = self.sentence_model.encode(
            sentences, 
            batch_size=16,  # Controlled batch processing
            show_progress_bar=True
        )
        
        qa_pairs = []
        processed_indices = set()
        
        for i in tqdm(range(len(embeddings)), desc="Generating Q&A Pairs"):
            if i in processed_indices:
                continue
            
            current_group = [sentences[i]]
            current_embedding = embeddings[i]
            
            # Find similar sentences
            for j in range(i + 1, len(embeddings)):
                if j in processed_indices:
                    continue
                
                similarity = cosine_similarity(
                    [current_embedding], 
                    [embeddings[j]]
                )[0][0]
                
                if similarity > self.similarity_threshold:
                    current_group.append(sentences[j])
                    processed_indices.add(j)
            
            # Process groups with multiple sentences
            if len(current_group) > 1:
                question = self.generate_questions(current_group)
                context = " ".join(current_group)
                qa_pairs.append((question, context))
                
                # Free up memory
                del current_group, context, question
                gc.collect()
        
        return qa_pairs
    
    def safe_write_csv(self, qa_pairs, output_path='qa_pairs.csv'):
        """Safely write Q&A pairs to CSV with robust error handling"""
        try:
            # Use csv module for more control
            with open(output_path, 'w', newline='', encoding='utf-8') as csvfile:
                csv_writer = csv.writer(
                    csvfile, 
                    quoting=csv.QUOTE_ALL,  # Quote all fields
                    escapechar='\\',  # Escape problematic characters
                    delimiter=','
                )
                
                # Write header
                csv_writer.writerow(['Question', 'Context'])
                
                # Write data rows
                for question, context in tqdm(qa_pairs, desc="Writing to CSV"):
                    # Sanitize text to remove any problematic characters
                    safe_question = self._sanitize_text(question)
                    safe_context = self._sanitize_text(context)
                    
                    csv_writer.writerow([safe_question, safe_context])
            
            print(f"Successfully saved {len(qa_pairs)} Q&A pairs to {output_path}")
        
        except Exception as e:
            print(f"Error writing to CSV: {e}")
            
            # Fallback: write to text file if CSV fails
            self._write_to_txt(qa_pairs, output_path.replace('.csv', '.txt'))
    
    def _sanitize_text(self, text):
        """Remove or escape problematic characters"""
        # Remove non-printable characters
        text = ''.join(char for char in text if char.isprintable())
        
        # Remove newlines and extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text
    
    def _write_to_txt(self, qa_pairs, output_path):
        """Fallback method to write to text file"""
        try:
            with open(output_path, 'w', encoding='utf-8') as f:
                for i, (question, context) in enumerate(qa_pairs, 1):
                    f.write(f"Q{i}: {question}\n")
                    f.write(f"Context{i}: {context}\n\n")
            print(f"Fallback: Saved Q&A pairs to text file {output_path}")
        except Exception as e:
            print(f"Error writing to text file: {e}")
    
    def generate_and_save_qa(self, output_path='qa_pairs.csv'):
        """End-to-end Q&A generation with memory management"""
        # Extract and preprocess text
        text = self.extract_text_from_pdf()
        sentences = self.split_sentences(text)
        
        # Generate Q&A pairs
        qa_pairs = self.generate_context_qa_pairs(sentences)
        
        # Save to CSV with robust error handling
        self.safe_write_csv(qa_pairs, output_path)
        
        # Cleanup
        del text, sentences, qa_pairs
        gc.collect()

# Usage example
def main():
    pdf_path = "/kaggle/input/attention-is-all-you-need/7181-attention-is-all-you-need.pdf"
    
    # Memory-efficient setup
    torch.cuda.empty_cache()
    generator = MemoryEfficientQAGenerator(pdf_path)
    generator.generate_and_save_qa()

if __name__ == "__main__":
    main()

In [ ]:
import os
import gc
import csv
import pandas as pd
import keras_nlp
import tensorflow as tf

class MemoryEfficientQAGenerator:
    def __init__(self, 
                 qa_csv_path, 
                 gemma_model_path,
                 output_path='qa_pairs_with_answers101.csv',
                 batch_size=4):
        """
        Ultra memory-efficient Q&A answer generator
        
        :param qa_csv_path: Path to input Q&A CSV
        :param gemma_model_path: Path to Gemma model weights
        :param output_path: Path for output CSV
        :param batch_size: Number of Q&A pairs to process simultaneously
        """
        # Limit GPU memory growth
        gpus = tf.config.experimental.list_physical_devices('GPU')
        if gpus:
            try:
                for gpu in gpus:
                    tf.config.experimental.set_memory_growth(gpu, True)
            except RuntimeError as e:
                print(f"Memory growth setting error: {e}")
        
        # Reduce TensorFlow logging
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
        
        # Configuration
        self.qa_csv_path = qa_csv_path
        self.output_path = output_path
        self.batch_size = batch_size
        
        # Load model with minimal memory footprint
        self.model = self._load_model(gemma_model_path)
    
    def _load_model(self, weights_path):
        """
        Load Gemma model with minimal memory usage
        """
        try:
            # Load base model architecture
            model = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")
            
            # Configure sampler
            sampler = keras_nlp.samplers.TopKSampler(k=5, seed=42)
            model.compile(sampler=sampler)
            
            # Load specific weights
            model.load_weights(weights_path)
            
            return model
        except Exception as e:
            print(f"Model loading error: {e}")
            return None
    
    def _create_prompt(self, context, question):
        """
        Create standardized prompt for answer generation
        """
        return f"""Instruction: Generate a concise answer to the question using the provided context. 
Provide only the direct answer without additional explanation.

Context:
{context}

Question:
{question}

Answer:
"""
    
    def generate_answers(self):
        """
        Generate answers in memory-efficient batches
        """
        # Read input CSV
        try:
            df = pd.read_csv(self.qa_csv_path)
        except Exception as e:
            print(f"CSV reading error: {e}")
            return
        
        # Prepare output storage
        results = []
        
        # Process in batches
        for i in range(0, len(df), self.batch_size):
            batch = df.iloc[i:i+self.batch_size]
            
            for _, row in batch.iterrows():
                try:
                    # Create prompt
                    prompt = self._create_prompt(
                        row['Context'], 
                        row['Question']
                    )
                    
                    # Generate answer
                    answer = self.model.generate(
                        prompt, 
                        max_length=200
                    )
                    
                    # Store results
                    results.append({
                        'Question': row['Question'],
                        'Context': row['Context'],
                        'Answer': answer.strip()
                    })
                    
                    # Manual garbage collection
                    gc.collect()
                    
                except Exception as e:
                    print(f"Answer generation error: {e}")
                    results.append({
                        'Question': row['Question'],
                        'Context': row['Context'],
                        'Answer': 'Generation Failed'
                    })
        
        # Create output DataFrame
        output_df = pd.DataFrame(results)
        
        # Save to CSV
        try:
            output_df.to_csv(self.output_path, index=False)
            print(f"Successfully saved {len(results)} Q&A pairs")
        except Exception as e:
            print(f"CSV writing error: {e}")
        
        # Final cleanup
        del output_df
        gc.collect()

def main():
    # Paths configuration
    qa_csv_path = 'qa_pairs.csv'
    gemma_model_path = '/kaggle/input/gemma-quizard-weights/keras/default/1/QA-Gemma-Quizard.weights.h5'
    output_path = 'qa_pairs_with_answers.csv'
    
    # Initialize generator
    generator = MemoryEfficientQAGenerator(
        qa_csv_path, 
        gemma_model_path, 
        output_path,
        batch_size=4  # Adjust based on your memory constraints
    )
    
    # Generate answers
    generator.generate_answers()

if __name__ == "__main__":
    main()

In [1]:
import os
import gc
import csv
import pandas as pd
import keras_nlp
import tensorflow as tf

class MemoryEfficientQAGenerator:
    def __init__(self, 
                 qa_csv_path, 
                 gemma_model_path,
                 output_path='qa_pairs_with_answers.csv',
                 batch_size=4):
        """
        Ultra memory-efficient Q&A answer generator
        
        :param qa_csv_path: Path to input Q&A CSV
        :param gemma_model_path: Path to Gemma model weights
        :param output_path: Path for output CSV
        :param batch_size: Number of Q&A pairs to process simultaneously
        """
        # Limit GPU memory growth
        gpus = tf.config.experimental.list_physical_devices('GPU')
        if gpus:
            try:
                for gpu in gpus:
                    tf.config.experimental.set_memory_growth(gpu, True)
            except RuntimeError as e:
                print(f"Memory growth setting error: {e}")
        
        # Reduce TensorFlow logging
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
        
        # Configuration
        self.qa_csv_path = qa_csv_path
        self.output_path = output_path
        self.batch_size = batch_size
        
        # Load model with minimal memory footprint
        self.model = self._load_model(gemma_model_path)
    
    def _load_model(self, weights_path):
        """
        Load Gemma model with minimal memory usage
        """
        try:
            # Load base model architecture
            model = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")
            
            # Configure sampler
            sampler = keras_nlp.samplers.TopKSampler(k=5, seed=42)
            model.compile(sampler=sampler)
            
            # Load specific weights
            model.load_weights(weights_path)
            
            return model
        except Exception as e:
            print(f"Model loading error: {e}")
            return None
    
    def _create_prompt(self, context, question):
        """
        Create standardized prompt for answer generation
        """
        return f"""Instruction: Generate a concise answer to the question using the provided context. 
Provide only the direct answer without additional explanation.

Context:
{context}

Question:
{question}

Answer:
"""
    
    def generate_answers(self):
        """
        Generate answers in memory-efficient batches
        """
        if not self.model:
            print("Model not loaded. Aborting generation.")
            return

        try:
            # Read the input CSV
            df = pd.read_csv(self.qa_csv_path)
            if not {'Context', 'Question'}.issubset(df.columns):
                print("Input CSV must contain 'Context' and 'Question' columns.")
                return
        except Exception as e:
            print(f"CSV reading error: {e}")
            return

        results = []
        for i in range(0, len(df), self.batch_size):
            batch = df.iloc[i:i + self.batch_size]
            for _, row in batch.iterrows():
                try:
                    # Create the prompt
                    prompt = self._create_prompt(row['Context'], row['Question'])

                    # Log the prompt being sent to the model
                    print(f"\nPrompt being sent to the model:\n{prompt}")g

                    # Generate the answer
                    answer = self.model.generate(prompt, max_length=200).strip()

                    # Store the result
                    results.append({
                        'Question': row['Question'],
                        'Context': row['Context'],
                        'Answer': answer
                    })

                except Exception as e:
                    print(f"Error at batch {i}, row {_}: {e}")
                    results.append({
                        'Question': row['Question'],
                        'Context': row['Context'],
                        'Answer': 'Generation Failed'
                    })

            # Perform garbage collection after processing the batch
            gc.collect()

        # Save results to the output CSV
        try:
            pd.DataFrame(results).to_csv(self.output_path, index=False)
            print(f"Successfully saved {len(results)} Q&A pairs.")
        except Exception as e:
            print(f"CSV writing error: {e}")

def main():
    # Paths configuration
    qa_csv_path = 'qa_pairs.csv'
    gemma_model_path = '/kaggle/input/gemma-quizard-weights/keras/default/1/QA-Gemma-Quizard.weights.h5'
    output_path = 'qa_pairs_with_answers.csv'
    
    # Initialize generator
    generator = MemoryEfficientQAGenerator(
        qa_csv_path, 
        gemma_model_path, 
        output_path,
        batch_size=4  # Adjust based on your memory constraints
    )
    
    # Generate answers
    generator.generate_answers()

if __name__ == "__main__":
    main()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 210 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



Prompt being sent to the model:
Instruction: Generate a concise answer to the question using the provided context. 
Provide only the direct answer without additional explanation.

Context:
Attention Is All You Need Ashish Vaswani Google Brain avaswani@google.comNoam Shazeer Google Brain noam@google.comNiki Parmar Google Research nikip@google.comJakob Uszkoreit Google Research usz@google.com Llion Jones Google Research llion@google.comAidan N. Gomezy University of Toronto aidan@cs.toronto.eduŁukasz Kaiser Google Brain lukaszkaiser@google.com Illia Polosukhinz illia.polosukhin@gmail.com Abstract The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. 3 Model Architecture Most competitive neural sequence transduction models have an encoder-decoder structure [ 5,2,29]. 7 Conclusion In this work, we presented the Transformer, the ﬁrst sequence transduction model based entirely on attention, replacing t

I0000 00:00:1734185029.455250     402 service.cc:145] XLA service 0x7ee0dafa1010 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734185029.455343     402 service.cc:153]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1734185029.741994     402 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Prompt being sent to the model:
Instruction: Generate a concise answer to the question using the provided context. 
Provide only the direct answer without additional explanation.

Context:
The best performing models also connect the encoder and decoder through an attention mechanism. Numerous efforts have since continued to push the boundaries of recurrent language models and encoder-decoder architectures [31, 21, 13]. Attention mechanisms have become an integral part of compelling sequence modeling and transduc- tion models in various tasks, allowing modeling of dependencies without regard to their distance in the input or output sequences [ 2,16]. The Transformer follows this overall architecture using stacked self-attention and point-wise, fully connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1, respectively. We also modify the self-attention sub-layer in the decoder stack to prevent positions from attending to subsequent positions. 3

KeyboardInterrupt: 

In [10]:
import os
import gc
import csv
import pandas as pd
import keras_nlp
import tensorflow as tf

class MemoryEfficientQAGenerator:
    def __init__(self, 
                 qa_csv_path, 
                 gemma_model_path,
                 output_path='/kaggle/working/qa_pairs_with_answers.csv',
                 batch_size=4):
        # Limit GPU memory growth
        gpus = tf.config.experimental.list_physical_devices('GPU')
        if gpus:
            try:
                for gpu in gpus:
                    tf.config.experimental.set_memory_growth(gpu, True)
            except RuntimeError as e:
                print(f"Memory growth setting error: {e}")
        
        # Reduce TensorFlow logging
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
        
        # Configuration
        self.qa_csv_path = qa_csv_path
        self.output_path = output_path
        self.batch_size = batch_size
        
        # Load model with minimal memory footprint
        self.model = self._load_model(gemma_model_path)
    
    def _load_model(self, weights_path):
        """
        Load Gemma model with minimal memory usage
        """
        try:
            # Load base model architecture
            model = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")
            
            # Configure sampler
            sampler = keras_nlp.samplers.TopKSampler(k=5, seed=42)
            model.compile(sampler=sampler)
            
            # Load specific weights
            model.load_weights(weights_path)
            
            return model
        except Exception as e:
            print(f"Model loading error: {e}")
            return None
    
    def _create_prompt(self, context, question):
        """
        Create standardized prompt for answer generation
        """
        return f"""Instruction: Generate a concise answer to the question using the provided context. 
Provide only the direct answer without additional explanation.

Context:
{context}

Question:
{question}

Answer:
"""
    
    def generate_answers(self):
        """
        Generate answers in memory-efficient batches
        """
        if not self.model:
            print("Model not loaded. Aborting generation.")
            return

        try:
            # Open the output CSV file for writing
            with open(self.output_path, 'w', newline='', encoding='utf-8') as csvfile:
                # Create CSV writer
                csv_writer = csv.writer(csvfile)
                # Write headers
                csv_writer.writerow(['Question', 'Context', 'Answer'])
                
                # Read input CSV
                df = pd.read_csv(self.qa_csv_path)
                if not {'Context', 'Question'}.issubset(df.columns):
                    print("Input CSV must contain 'Context' and 'Question' columns.")
                    return

                # Process each row
                for index, row in df.iterrows():
                    try:
                        # Create the prompt
                        prompt = self._create_prompt(row['Context'], row['Question'])

                        # Print the prompt details
                        print(f"\n--- PROMPT {index + 1} ---")
                        print(f"Context: {row['Context']}")
                        print(f"Question: {row['Question']}")

                        # Generate the answer
                        answer = self.model.generate(prompt, max_length=200)
                        
                        # Print the generated answer
                        print("\n--- ANSWER ---")
                        print(answer)
                        print("-" * 50)

                        # Write to CSV immediately
                        csv_writer.writerow([
                            row['Question'], 
                            row['Context'], 
                            answer
                        ])
                        csvfile.flush()  # Ensure writing to disk

                        # Aggressive memory clearing
                        del answer
                        gc.collect()
                        tf.keras.backend.clear_session()

                    except Exception as e:
                        print(f"Error processing row {index}: {e}")
                        
                        # Write error to CSV
                        csv_writer.writerow([
                            row['Question'], 
                            row['Context'], 
                            'Generation Failed'
                        ])
                        csvfile.flush()

        except Exception as e:
            print(f"CSV processing error: {e}")

def main():
    # Paths configuration
    qa_csv_path = '/kaggle/input/qa-pairs-attention-paper/qa_pairs.csv'
    gemma_model_path = '/kaggle/input/gemma-quizard-weights/keras/default/1/QA-Gemma-Quizard.weights.h5'
    output_path = 'qa_pairs_with_answers.csv'
    
    # Initialize generator
    generator = MemoryEfficientQAGenerator(
        qa_csv_path, 
        gemma_model_path, 
        output_path,
        batch_size=4  # Adjust based on your memory constraints
    )
    
    # Generate answers
    generator.generate_answers()

if __name__ == "__main__":
    main()

/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 210 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



--- PROMPT 1 ---
Context: Attention Is All You Need Ashish Vaswani Google Brain avaswani@google.comNoam Shazeer Google Brain noam@google.comNiki Parmar Google Research nikip@google.comJakob Uszkoreit Google Research usz@google.com Llion Jones Google Research llion@google.comAidan N. Gomezy University of Toronto aidan@cs.toronto.eduŁukasz Kaiser Google Brain lukaszkaiser@google.com Illia Polosukhinz illia.polosukhin@gmail.com Abstract The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. 3 Model Architecture Most competitive neural sequence transduction models have an encoder-decoder structure [ 5,2,29]. 7 Conclusion In this work, we presented the Transformer, the ﬁrst sequence transduction model based entirely on attention, replacing the recurrent layers most commonly used in encoder-decoder architectures with multi-headed self-attention.
Question: What are the dominant sequence transduction mod